<a href="https://colab.research.google.com/github/tarunsha009/LLM-Learning/blob/main/learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
!pip install --upgrade fsspec datasets

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets)  # See the structure
print("\nFirst training example:", raw_datasets["train"][0])

In [ ]:
print("\nFirst training example:", raw_datasets["test"][0])

In [ ]:
print("\nFirst training example:", raw_datasets["validation"][0])


In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding="max_length")

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
print(tokenized_datasets)


In [ ]:
sample = raw_datasets['train'][0]
print(sample.keys())
tokenize_sample = tokenizer(sample["sentence1"], sample["sentence2"], truncation=True, padding="max_length")
print("Tokenized iutput keys:", tokenize_sample.keys())
print("\nTokenized sentence 1:", tokenize_sample["input_ids"])
print("\nDecoded sentence 1:", tokenizer.decode(tokenize_sample["input_ids"]))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
print(model)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my_awesome_model_pari",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01)

In [ ]:
from transformers import Trainer

trainee = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

trainee.train()

In [ ]:
results = trainee.evaluate()
print(results)

In [ ]:
model.save_pretrained("bert-mrpc-finetuned")
tokenizer.save_pretrained("bert-mrpc-finetuned")